# Applying a ConvLSTM network on the MC16_2 dataset

This code extracts and processes video data from Dropbox, moves it to Google Drive, and builds a deep learning model to classify the videos. Frames are extracted from each video, resized, and padded if necessary. Labels are encoded, and the dataset is split into training and testing sets. A ConvLSTM network, combining convolutional layers and LSTM layers, is built to capture both spatial and temporal features. The model is trained on the dataset and evaluated to determine its accuracy in classifying the videos based on predefined labels. This approach is useful for tasks involving video classification and activity recognition.

![flowchart](Figures/week5-flowchart.png)

## Phase I: Setting Up rClone

The following code performs several steps related to installing and configuring `rclone`, a command-line tool for managing files on cloud storage. Here is a step-by-step explanation of what each part of the code does:

1.   **Install `rclone` and other necessary tools:**

  *   The `wget` command downloads the latest version of `rclone` for Linux in a zip file.
  *   The `unzip` command extracts the contents of the zip file.
  *   The `mkdir -p ~/bin` command creates a `bin` in the home directory if it doesn't already exist.
  *   The `mv` command moves the `rclone` executable to the `~bin` directory.



In [ ]:
# Install rclone and other necessary tools
!wget https://downloads.rclone.org/rclone-current-linux-amd64.zip -O rclone.zip
!unzip rclone.zip
!mkdir -p ~/bin
!mv rclone-*-linux-amd64/rclone ~/bin/

2.   **Update the PATH environment variable:**

  *   The `os.environ["PATH"]` line appends the `~/bin` directory to the PATH environment variable, ensuring that the `rclone` command can be executed from anywhere.





In [ ]:
import os

# Add the ~/bin directory to the PATH environment variable
os.environ["PATH"] += os.pathsep + os.path.expanduser("~/bin")

3.   **Verify `rclone` installation:**

  *   This command checks if `rclone` has been successfully installed by printing its version.




In [ ]:
# Verify rclone is in the PATH
!rclone version

4.   **Configure `rclone` with Dropbox:**

  *   The `config_content` variable contains the configuration for a Dropbox remote named `cichlidVideo`.
  *   The `with open('rclone.conf', 'w') as config_file:` line writes this configuration to an `rclone.conf` file.

In [ ]:
config_content = """
[cichlidVideo]
type = dropbox
token = {"access_token":"---","token_type":"---","expiry":"---"}
"""

# Write the content to the rclone.conf file
with open('rclone.conf', 'w') as config_file:
    config_file.write(config_content)

5.   **Mount Google Drive:**

  *   This code mounts Google Drive to the `/content/drive` directory, making it accessible within the Colab environment.

  This setup allows you to use `rclone` to manage files between local storage, Dropbox, and Google Drive within a Google Colab environment.

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

## Phase II: Moving Files Over From Dropbox

The following code performs several tasks related to copying, extracting, and processing video clips and their labels from Dropbox to Google Drive, and then verifying and preparing these files for further analysis. Here is a step-by-step explanation of what each part of the code does:

1.   **Import the `tarfile` module:**

In [ ]:
import tarfile

2. **Copy the `.tar` file from Dropbox to Google Drive:**

In [ ]:
# Copy the .tar file from DropBox to Google Drive
source_path = "BioSci-McGrath/Apps/CichlidPiData/__AnnotatedData/LabeledVideos/Clips/MC16_2.tar"
dest_path = "/content/drive/MyDrive/OMSCS/Cichlids/"
!rclone --config rclone.conf -v copy cichlidVideo:{source_path} {dest_path}

3. **Extract the `.tar` file in Google Drive:**

In [ ]:
tar_file_path = "/content/drive/MyDrive/OMSCS/Cichlids/MC16_2.tar"
dest_path_extract = "/content/drive/MyDrive/OMSCS/Cichlids/unzipped"

# Open and extract the tar file
with tarfile.open(tar_file_path, 'r') as tar:
  tar.extractall(path=dest_path_extract)

4. **Verify the extraction:**

In [ ]:
# Verify the extraction
!ls -l /content/drive/MyDrive/OMSCS/Cichlids/unzipped

5. **Copy the CSV file with manual labels from Dropbox to Google Drive:**

In [ ]:
source_labels = "BioSci-McGrath/Apps/CichlidPiData/__AnnotatedData/LabeledVideos/ManualLabels.csv"
dest_path_labels = "/content/drive/MyDrive/OMSCS/Cichlids/"
!rclone --config rclone.conf -v copy cichlidVideo:{source_labels} {dest_path_labels}

6. **Read the CSV file into a DataFrame:**

In [ ]:
import pandas as pd

# Path to the CSV file on Google Drive
csv_path = "/content/drive/MyDrive/OMSCS/Cichlids/ManualLabels.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path)

7. **Add the full path to each file in the DataFrame:**

In [ ]:
# Adding the full path to each file
base_path = "/content/drive/MyDrive/OMSCS/Cichlids/unzipped/MC16_2/"
df['full_path'] = base_path + df['ClipName']

# Print the DataFrame
#print(df)

8. **Print the unique manual labels (commented out):**

In [ ]:
unique_manual_labels = df['ManualLabel'].unique()
#print(unique_manual_labels)

9. **Verify the files in the target directory:**

In [ ]:
import os

# Path to the target directory on Google Drive
target_dir = "/content/drive/MyDrive/OMSCS/Cichlids/unzipped/MC16_2"

# List all files in the target directory
files_in_directory = os.listdir(target_dir)

# Display the first few files to verify
#print(files_in_directory[:5])

10. **Prepare the list of files to process with their corresponding manual labels:**

In [ ]:
# Extract the ClipName column from the DataFrame
clip_names = df['ClipName'].tolist()
manual_labels = df['ManualLabel'].tolist()

files_to_process = []

# Check which ClipName values are present in the directory
for file in files_in_directory:
    for clip_name, manual_label in zip(clip_names, manual_labels):
      if clip_name in file:
        #files_to_process.append([file for clip in clip_names if clip in file])
        files_to_process.append({'file': base_path + file, 'manual_label': manual_label})

11. **Display the files to process with their corresponding manual labels (commented out):**

In [ ]:
# Display the files to process with their corresponding manual labels
"""
print("Files to process with Manual Labels:")
for item in files_to_process:
    print(f"File: {item['file']}, Manual Label: {item['manual_label']}")
"""

12. **Print the first file to process and the total number of files:**

In [ ]:
print(files_to_process[0])
print(f"Number of files: {len(files_to_process)}")

The above setup allows you to efficiently manage and process video clips and their labels from Dropbox to Google Drive with a Google Colab environment, preparing them for subsequent analysis or processing steps.

## Phase III: Training The Model

This code performs a sequence of tasks to load and preprocess video data, build a neural network model, train it, and evaluate its performance. Below is a detailed step-by-step breakdown:

1. **Loading the Libraries:**

  *   This section imports necessary libraries for file handling, image processing, numerical operations, data manipulation, and building a deep learning model.



In [ ]:
# Loading the libraries
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam

2. **Set Parameters:**

  *   These variables define the dimensions of frames extracted from videos and the number of frames to be extracted from each video.



In [ ]:
frame_width = 64
frame_height = 64
num_frames = 10

3. **Function to Extract Frames from Videos:**

  *   This function reads a video file, resizes the frames to the specified dimensions, and collects a fixed number of frames. If the video has fewer frames that required, it pads the sequence with black frames.

In [ ]:
# Extract frames from videos
def extract_frames(video_path, frame_width, frame_height, num_frames):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while len(frames) < num_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_width, frame_height))
        frames.append(frame)
    cap.release()
    # If the video has fewer frames, pad with black frames
    while len(frames) < num_frames:
        frames.append(np.zeros((frame_height, frame_width, 3), dtype=np.uint8))
    return np.array(frames[:num_frames])

4. **Function to Create the Dataset:**

  *   This function preprocesses a list of video files and their corresponding labels, extracting frames from each video and storing them along with their labels.

In [ ]:
def create_dataset(files_to_process, frame_width, frame_height, num_frames):
    X = []
    y = []
    for file_info in files_to_process:
        video_path = file_info['file']
        label = file_info['manual_label']
        frames = extract_frames(video_path, frame_width, frame_height, num_frames)
        X.append(frames)
        y.append(label)
    return np.array(X), np.array(y)

5. **Example Usage to Create Dataset:**

  *   Here, the dataset is created by calling the `create_dataset` function with the list of files to process.

In [ ]:
# Example usage
X, y = create_dataset(files_to_process, frame_width, frame_height, num_frames)

6. **Encode Labels:**

  *   Labels are mapped to numerical values and then converted to categorical format using one-hot-encoding.

In [ ]:
label_mapping = {'m': 0, 'o': 1, 'c': 2, 'p': 3, 'd': 4, 'x': 5, 'f': 6, 't': 7, 'b': 8, 's': 9}
y = np.array([label_mapping[label] for label in y])
y = to_categorical(y)

7. **Split Dataset into Training and Testing Sets:**

  *   The dataset is split into training and testing sets with 80% of the data used for training and 20% for testing.



In [ ]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

8. **Build the Model:**
  * The model is defined as a sequential stack of layers:
    * `TimeDistributed` wrapper to apply the same convolutional and pooling operations to each frame.
    * An LSTM layer to process the sequences of frames.
    * A dense layer with softmax activation to output class probabilities.

In [ ]:
# Build model
model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu'), input_shape=(num_frames, frame_height, frame_width, 3)),
    TimeDistributed(MaxPooling2D((2, 2))),
    TimeDistributed(Flatten()),
    LSTM(64, activation='relu'),
    Dense(len(label_mapping), activation='softmax')
])

9. **Compile the Model:**

  * The model is compiled with Adam optimizer and categorical cross-entropy loss, with accuracy as a metric.

In [ ]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

10. **Train the Model:**

  * The model is trained on the training data for 100 epochs, using the test data for validation.

In [ ]:
# Train model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

11. **Evaluate the Model:**

  * The model's performance is evaluated on the test set, and the test accuracy is printed.

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy}')

This comprehensive process involves extracting frames from videos, preparing the dataset, building a deep learning model, and evaluating its performance, all within a Google Colab environment.